In [0]:
# Set the source information
account_name = "medallion1"
container_name = "bronze"
folder_path = "20240118"   # <-- Set the name of your Parquet folder here

# Provide your storage account key 
storage_account_access_key = "+Z7unWQrXCsx8I2j1TggARQXoFA0vWOeSGD4YilMjopyDkoPX+E9SuJKMk6DCo+mW7Yh/CM69VEx+AStgJIQ4Q==" # Replace with your key

# Unmount the directory if it is already mounted
if f"/mnt/{container_name}" in [mnt.mountPoint for mnt in dbutils.fs.mounts()]:
  dbutils.fs.unmount(f"/mnt/{container_name}")

# Mount the Blob Storage account to a DBFS directory with credentials
dbutils.fs.mount(
  source=f"wasbs://{container_name}@{account_name}.blob.core.windows.net/",
  mount_point=f"/mnt/{container_name}",
  extra_configs={
    f"fs.azure.account.key.{account_name}.blob.core.windows.net":storage_account_access_key
  }
)

# List all Parquet files in the specified folder on the Blob Storage account
file_paths = [f"/mnt/{container_name}/{folder_path}/{file.name}"
              for file in dbutils.fs.ls(f"/mnt/{container_name}/{folder_path}")
             if file.isFile() and file.name.endswith(".parquet")]
             
# Create the database using spark.sql()
spark.sql(f"CREATE DATABASE IF NOT EXISTS bronze")

# Load each Parquet file into a separate DataFrame in Databricks
for file_path in file_paths:
    # Derive the table name from the file name
    table_name = file_path.split("/")[-1].split(".")[0]
    # Read the Parquet file into a DataFrame
    df = spark.read.parquet(file_path) 
    # Create table using Hive metastore
    df.write.format("parquet").saveAsTable(f"bronze.{table_name}")

# Unmount the Blob Storage account om the DBFS directory
dbutils.fs.unmount(f"/mnt/{container_name}")